In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import sys
from pandas import HDFStore,DataFrame

import matplotlib.pyplot as plt
#from matplotlib_venn import venn2


# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

pd.set_option('display.float_format', lambda x: '%.2f' % x)


In [3]:
hdf = HDFStore('../data/hdf/datastore.h5')

print(hdf.keys())
hdf.close()

['/Account_Assignment', '/DataCR_from_2015_Cancellations_Source_Systems', '/DataCRjournals', '/ECH Customer Data', '/NPS_Cleansed_Data', '/Product_Assignment_2019', '/SIS_ECRID_Mapping', '/churn_activities', '/churn_products', '/churn_risks_V02', '/churn_vps', '/interaction', '/sis_mapping', '/usage']


In [4]:
journals_contracts = pd.read_hdf('../data/hdf/datastore.h5', 'DataCRjournals', )

(8288244, 36)

,Agreement End Date,Agreement Number,Agreement Start Date,Bookigns - Committed Print(Rep),Bookings - Final Net Price - Agent Discount Amount(Rep),Business Division (Agreement SIS),Business Indicator,Calculated New/Renewal,Country Name (Agreement SIS),Division,...,Saleable Product Name (Source),Sales Division (Agreement SIS),Sales Type,Status,Status Change Date,Subregion Grouping,Subscription End Date,Subscription Start Date,Subscription Start Year,WIP Flag
0,2015-12-31,DLT1,2015-01-01,0.00,91.31,HS EMEA/LA,nan,Renewal,Netherlands,Global,...,RADIOTHERAPY & ONCOLOGY,HS Europe North,MS,N.A,0-00-00,Global Member,2015-12-31,2015-01-01,2015,INVOICED
1,2015-12-31,DLT1,2015-01-01,0.00,182.62,HS EMEA/LA,nan,Renewal,Netherlands,Global,...,RADIOTHERAPY & ONCOLOGY,HS Europe North,MS,N.A,0-00-00,Global Member,2015-12-31,2015-01-01,2015,INVOICED
2,2015-12-31,DLT1,2015-01-01,0.00,43.88,HS EMEA/LA,nan,Renewal,Netherlands,Global,...,RADIOTHERAPY & ONCOLOGY ONLINE (PHOENIX),HS Europe North,MS,N.A,0-00-00,Global Member,2015-12-31,2015-01-01,2015,INVOICED
3,2015-12-31,DLF1,2015-01-01,0.00,445.15,A&G_MAEU,nan,Renewal,Netherlands,A&G,...,EUROPEAN GERIATRIC MEDICINE,AG-MAEU-North_Europe0-01D,LF,N.A,0-00-00,Region North Europe,2015-12-31,2015-01-01,2015,INVOICED
4,2015-12-31,DLF1,2015-01-01,0.00,183.47,A&G_MAEU,nan,Renewal,Netherlands,Global,...,ANNALES DE DERMATOLOGIE,AG-MAEU-North_Europe0-01D,MS,N.A,0-00-00,Global Member,2015-12-31,2015-01-01,2015,INVOICED


In [4]:
# Define CRM and UPL contracts
crm_list = ['Active Renewal', 'Cancelled', 'Evegreen Renewal', 'Merged', 'New', 'Standard Renewal']
upl_pattern = 'UPL-5'

In [5]:
# Read Data
ECH = pd.read_pickle('../data/pickle/ECH Customer Data.pickle')

journals_contracts = pd.read_pickle('../data/pickle/DataCRjournals.pickle')
other_contracts = pd.read_pickle('../data/pickle/DataCRother.pickle')
# churn_activities = pd.read_pickle('../data/pickle/191212_churn_activities.pickle')
# churn_risks = pd.read_pickle('../data/pickle/191224_churn_risks_V02.pickle')
# account_assignment = pd.read_pickle('../data/pickle/Account_Assignment.pickle')
# NPS = pd.read_pickle('../data/pickle/NPS_Cleansed_Data.pickle')
# product_assignment = pd.read_pickle('../data/pickle/Product_Assignment_2019.pickle')
# usage = pd.read_pickle('../data/pickle/usage.pickle')
# interactions = pd.read_pickle('../data/pickle/interaction.pickle')
cancellations = pd.read_pickle('../data/pickle/DataCR_from_2015_Cancellations_Source_Systems.pickle')

sis_ecr_mapping = pd.read_pickle('../data/pickle/SIS_ECRID_Mapping.pickle')


 
### KEEP ONLY CRM AND UPL DATA

### Remove non-RSS projects

In [6]:
is_crm_journals = journals_contracts['Sales Type'].isin(crm_list)
is_upl_journals = journals_contracts['Agreement Number'].str.startswith(upl_pattern).fillna(False)
print(is_crm_journals.value_counts())
print(is_upl_journals.value_counts())

True     5061850
False    3226394
Name: Sales Type, dtype: int64
False    8287672
True         572
Name: Agreement Number, dtype: int64


In [7]:
# Keep only crm or upl agreements
journals_contracts = journals_contracts[is_crm_journals | is_upl_journals]

journals_contracts['Sales Type'].value_counts(dropna=False)

Standard Renewal    2862422
Active Renewal      2030248
New                  104056
Merged                63308
Cancelled              1816
NaN                     572
Name: Sales Type, dtype: int64

In [8]:
is_crm_others = other_contracts['Sales Type'].isin(crm_list)
is_upl_others = other_contracts['Agreement Number'].str.startswith(upl_pattern).fillna(False)
print(is_crm_others.value_counts())
print(is_upl_others.value_counts())

False    20423801
True      1545053
Name: Sales Type, dtype: int64
False    21948444
True        20410
Name: Agreement Number, dtype: int64


In [8]:
# Keep only crm or upl agreements
other_contracts = other_contracts[is_crm_others | is_upl_others]

other_contracts['Sales Type'].value_counts(dropna=False)

New                 498478
Active Renewal      187118
Standard Renewal    129593
Cancelled            23501
NaN                  10756
Merged                8841
Name: Sales Type, dtype: int64

In [14]:
sis_ecr_mapping['source_record_id'] = sis_ecr_mapping['source_record_id'].astype(str)
sis_ecr_mapping.dtypes

ecrid               object
source_record_id    object
dtype: object

In [15]:
x = pd.merge(journals_contracts, sis_ecr_mapping, how='left', left_on='SIS Id  (Agreement SIS)', right_on='source_record_id')

In [16]:
x.head()

,Agreement End Date,Agreement Number,Agreement Start Date,Bookigns - Committed Print(Rep),Bookings - Final Net Price - Agent Discount Amount(Rep),Business Division (Agreement SIS),Business Indicator,Calculated New/Renewal,Country Name (Agreement SIS),Division,...,Sales Type,Status,Status Change Date,Subregion Grouping,Subscription End Date,Subscription Start Date,Subscription Start Year,WIP Flag,ecrid,source_record_id
0,2016-12-31,1-7305688604,2014-01-01,0.00,3660.81,A&G_MAEU,Renewal,Renewal,Norway,A&G,...,Standard Renewal,Complete,2014-12-20,Region North Europe,2015-12-31,2015-01-01,2015,INVOICED,NaN,NaN
1,2016-12-31,1-7305688604,2014-01-01,0.00,1544.53,A&G_MAEU,Renewal,Renewal,Norway,A&G,...,Standard Renewal,Complete,2014-12-20,Region North Europe,2015-12-31,2015-01-01,2015,INVOICED,NaN,NaN
2,2016-12-31,1-7305688604,2014-01-01,0.00,2415.28,A&G_MAEU,Renewal,Renewal,Norway,A&G,...,Standard Renewal,Complete,2014-12-20,Region North Europe,2015-12-31,2015-01-01,2015,INVOICED,NaN,NaN
3,2016-12-31,1-7305688604,2014-01-01,0.00,1616.24,A&G_MAEU,Renewal,Renewal,Norway,A&G,...,Standard Renewal,Complete,2014-12-20,Region North Europe,2015-12-31,2015-01-01,2015,INVOICED,NaN,NaN
4,2016-12-31,1-7305688604,2014-01-01,0.00,3958.80,A&G_MAEU,Renewal,Renewal,Norway,A&G,...,Standard Renewal,Complete,2014-12-20,Region North Europe,2015-12-31,2015-01-01,2015,INVOICED,NaN,NaN


In [24]:
len(np.intersect1d(sis_ecr_mapping['source_record_id'], journals_contracts['SIS Id  (Agreement SIS)']))

0

In [22]:
x[['SIS Id  (Agreement SIS)','source_record_id']]

,SIS Id (Agreement SIS),source_record_id
0,416754,NaN
1,416754,NaN
2,416754,NaN
3,416754,NaN
4,416754,NaN
...,...,...
2531206,ECR-998,NaN
2531207,ECR-998,NaN
2531208,ECR-998,NaN
2531209,ECR-998,NaN


In [19]:
journals_contracts.columns

Index(['Agreement End Date', 'Agreement Number', 'Agreement Start Date',
       'Bookigns - Committed Print(Rep)',
       'Bookings - Final Net Price - Agent Discount Amount(Rep)',
       'Business Division (Agreement SIS)', 'Business Indicator',
       'Calculated New/Renewal', 'Country Name (Agreement SIS)', 'Division',
       'HQ SIS Id (Agreement SIS)', 'Invoice Date', 'Invoice Num',
       'Name  (Agreement SIS)', 'Parent Agreement Number', 'Payment Term',
       'Payment Term Description', 'Payment Term Type', 'Product Line Level 1',
       'Product Line Level 2', 'Product Line Level 3', 'Product Line Level 4',
       'Product Revenue Type', 'RSO', 'Renewal Exp Complete Date',
       'SIS Id  (Agreement SIS)', 'Saleable Product Name (Source)',
       'Sales Division (Agreement SIS)', 'Sales Type', 'Status',
       'Status Change Date', 'Subregion Grouping', 'Subscription End Date',
       'Subscription Start Date', 'Subscription Start Year', 'WIP Flag'],
      dtype='object')

In [11]:
sis_ecr_mapping.dtypes

ecrid               object
source_record_id     int64
dtype: object

In [96]:
cancellations.columns

Index(['Source System', 'SIS Id  (Agreement SIS)', 'Source System.1',
       'HQ SIS Id (Agreement SIS)', 'Name  (Agreement SIS)',
       'Business Division (Agreement SIS)', 'Sales Division (Agreement SIS)',
       'Division', 'RSO', 'Subregion Grouping', 'Country Name (Agreement SIS)',
       'WIP Flag', 'Status', 'Wip Type', 'Business Indicator', 'Sales Type',
       'Calculated New/Renewal', 'Payment Term', 'Payment Term Description',
       'Payment Term Type', 'Status Change Date', 'Reporting Year (2015)',
       'Renewal Exp Complete Date', 'Product Revenue Type',
       'Product Line Level 1', 'Product Line Level 2', 'Product Line Level 3',
       'Product Line Level 4', 'Saleable Product Name (Source)',
       'Agreement Number', 'Agreement Start Date', 'Agreement End Date',
       'Subscription Start Date', 'Subscription End Date',
       'Parent Agreement Number', 'Currency(Entered)',
       'Revenue Final Net Price - Agent Discount Amount(Rep)',
       'Revenue Committed Pr

### Pick Value from parent agreement

In [95]:
cancellations.head()

,Source System,SIS Id (Agreement SIS),Source System.1,HQ SIS Id (Agreement SIS),Name (Agreement SIS),Business Division (Agreement SIS),Sales Division (Agreement SIS),Division,RSO,Subregion Grouping,...,Subscription End Date,Parent Agreement Number,Currency(Entered),Revenue Final Net Price - Agent Discount Amount(Rep),Revenue Committed Print(Rep),Bookings - Final Net Price - Agent Discount Amount(Rep),Bookigns - Committed Print(Rep),Cancellation Month,Cancellation Reason,Cancellation Year
0,CRM,1060896,CRM,1060896,Hospital Infantil de las Californias,HS EMEA/LA,HS-EMEALA-RLAM0000000-01D,HS,EMEALA,HS RLAM,...,2016-05-14,NaN,USD,$nan,$nan,$0,$0,201702,Unknown,2017
1,CRM,1060896,CRM,1060896,Hospital Infantil de las Californias,HS EMEA/LA,HS-EMEALA-RLAM0000000-01D,HS,EMEALA,HS RLAM,...,2016-05-14,NaN,USD,$nan,$nan,$838,$0,201702,Unknown,2017
2,CRM,1060896,CRM,1060896,Hospital Infantil de las Californias,HS EMEA/LA,HS-EMEALA-RLAM0000000-01D,HS,EMEALA,HS RLAM,...,2016-05-14,NaN,USD,$nan,$nan,$0,$0,201702,Unknown,2017
3,CRM,1060896,CRM,1060896,Hospital Infantil de las Californias,HS EMEA/LA,HS-EMEALA-RLAM0000000-01D,HS,EMEALA,HS RLAM,...,2016-05-14,NaN,USD,$nan,$nan,$0,$0,201702,Unknown,2017
4,CRM,1060896,CRM,1060896,Hospital Infantil de las Californias,HS EMEA/LA,HS-EMEALA-RLAM0000000-01D,HS,EMEALA,HS RLAM,...,2016-05-14,NaN,USD,$nan,$nan,$1,$0,201702,Unknown,2017


In [92]:
ECH.head()

,ecrid,name,city,Country ISO,Region,consortium,post_code,Classification
0,ECR-560156,National Institute for Research-Development in...,Timişoara,RO,NaN,$nan,300569,Academic
1,ECR-10349400,Donostia International Physics Center,San Sebastian,ES,NaN,$nan,20018,Government
2,ECR-29390542,Twist Bioscience,San Francisco,US,CA,$nan,94158-2117,Corporate
3,ECR-10234417,University of Pennsylvania Department of Patho...,Philadelphia,US,PA,$nan,19104,Academic
4,ECR-576739,Meerut Institute of Engineering and Technology,Meerut,IN,NaN,$nan,250005,Academic


In [93]:
journals_contracts.head()

,Agreement End Date,Agreement Number,Agreement Start Date,Bookigns - Committed Print(Rep),Bookings - Final Net Price - Agent Discount Amount(Rep),Business Division (Agreement SIS),Business Indicator,Calculated New/Renewal,Country Name (Agreement SIS),Division,...,Saleable Product Name (Source),Sales Division (Agreement SIS),Sales Type,Status,Status Change Date,Subregion Grouping,Subscription End Date,Subscription Start Date,Subscription Start Year,WIP Flag
0,2015-12-31,DLT1,2015-01-01,$0,$91,HS EMEA/LA,NaN,Renewal,Netherlands,Global,...,RADIOTHERAPY & ONCOLOGY,HS Europe North,MS,N.A,0-00-00,Global Member,2015-12-31,2015-01-01,2015,INVOICED
1,2015-12-31,DLT1,2015-01-01,$0,$183,HS EMEA/LA,NaN,Renewal,Netherlands,Global,...,RADIOTHERAPY & ONCOLOGY,HS Europe North,MS,N.A,0-00-00,Global Member,2015-12-31,2015-01-01,2015,INVOICED
2,2015-12-31,DLT1,2015-01-01,$0,$44,HS EMEA/LA,NaN,Renewal,Netherlands,Global,...,RADIOTHERAPY & ONCOLOGY ONLINE (PHOENIX),HS Europe North,MS,N.A,0-00-00,Global Member,2015-12-31,2015-01-01,2015,INVOICED
3,2015-12-31,DLF1,2015-01-01,$0,$445,A&G_MAEU,NaN,Renewal,Netherlands,A&G,...,EUROPEAN GERIATRIC MEDICINE,AG-MAEU-North_Europe0-01D,LF,N.A,0-00-00,Region North Europe,2015-12-31,2015-01-01,2015,INVOICED
4,2015-12-31,DLF1,2015-01-01,$0,$183,A&G_MAEU,NaN,Renewal,Netherlands,Global,...,ANNALES DE DERMATOLOGIE,AG-MAEU-North_Europe0-01D,MS,N.A,0-00-00,Global Member,2015-12-31,2015-01-01,2015,INVOICED


In [86]:
cancellations['Cancellation Year'] = (cancellations['Cancellation Month']/100).astype(int)

NameError: name 'journals' is not defined

In [6]:
# customer ids cols
ECH_ecrid_col = 'ecrid'
journals_ecrid_col = 'SIS Id  (Agreement SIS)'
other_contracts_ecrid_col = 'SIS Id  (Agreement SIS)'
churn_activities_ecrid_col = 'ECR Id'
churn_risks_ecrid_col = 'Account Name: ECR Id'
account_assignment_ecrid_col = 'ECRID'
NPS_ecrid_col = 'ECR_ID'
usage_ecr_id = 'ECR_ID'
interactions_ecrid_col = 'ECR_ID'
cancellations_ecrid_col = 'SIS Id  (Agreement SIS)'

In [7]:
# Customers and contracts

ECH_customers = ECH[ECH_ecrid_col].unique()

journals_customers = journals_contracts[journals_ecrid_col].unique()
other_customers = other_contracts[other_contracts_ecrid_col].unique()
churn_activity_customers = churn_activities[churn_activities_ecrid_col].unique().astype(str)
churn_risks_customers = churn_risks[churn_risks_ecrid_col].unique().astype(str)
account_assignment_customers = account_assignment[account_assignment_ecrid_col].unique().astype(str)
NPS_customers = NPS[NPS_ecrid_col].unique().astype(str)
usage_customers = usage[usage_ecr_id].unique().astype(str)
interactions_customers = interactions[interactions_ecrid_col].unique().astype(str)
cancellation_customers = cancellations[cancellations_ecrid_col].unique().astype(str)

#venn2([set(['A', 'B', 'C', 'D']), set(['D', 'E', 'F'])])
#plt.show()


In [8]:
print(f'Unique customers in ECH {len(ECH_customers)}')
print(f'Unique customers in journals {len(journals_customers)}')
print(f'Unique customers in other_contracts {len(other_customers)}')
print(f'Unique customers in churn_activity_customers {len(churn_activity_customers)}')
print(f'Unique customers in churn_risks_customers {len(churn_risks_customers)}')
print(f'Unique customers in account_assignment_customers {len(account_assignment_customers)}')
print(f'Unique customers in NPS_customers {len(NPS_customers)}')
print(f'Unique customers in usage_customers {len(usage_customers)}')
print(f'Unique customers in interactions_customers {len(interactions_customers)}')
print(f'Unique customers in cancellations {len(cancellation_customers)}')



Unique customers in ECH 137546
Unique customers in journals 84056
Unique customers in other_contracts 34146
Unique customers in churn_activity_customers 25903
Unique customers in churn_risks_customers 40009
Unique customers in account_assignment_customers 68078
Unique customers in NPS_customers 8464
Unique customers in usage_customers 21127
Unique customers in interactions_customers 27590
Unique customers in cancellations 8586


### 1.3 million ECR-IDs in whole data. is it just customers in Sales Force

### Mapping IDs is best appoarch
### makeshift approach is to append ECR


### Account assignment 
cascades down not upwards
so if parent is assigned then all children area also assigned

Hierarchy Data:
Parent Child
A      X
A      Y
Y      Z
X      Y
X      Z
X      X

designed to match Parent column to available ECR-ID
column: Hierarcy Level - 1 for child 2 for grand child


In [32]:
# Common customers between ECH and journals

len(np.intersect1d(ECH_customers, journals_customers, assume_unique=True))

37228

In [63]:
len(np.setdiff1d(ECH_customers, journals_customers, assume_unique=True))

100318

In [64]:
len(np.setdiff1d(journals_customers, ECH_customers, assume_unique=True))

46828

In [65]:
joujournals_customers_not_in_ECH = np.setdiff1d(journals_customers, ECH_customers, assume_unique=True)

In [69]:
joujournals_customers_not_in_ECH[0:100]

array(['100034', '100097', '1001', '1003', '1004', '1005', '10058',
       '1006', '10062', '100710', '100789', '1009', '10092', '100925',
       '100929', '1011', '101204', '1013', '10136', '101366', '101395',
       '1014', '101478', '1015', '1017', '1017374', '1017377', '1017394',
       '1017395', '1017397', '1017398', '1017415', '1017417', '1017594',
       '1017595', '1017657', '1017658', '1017675', '1017685', '1017696',
       '1017756', '1017834', '1017840', '1017895', '1017934', '101799',
       '1018', '1018340', '101848', '1018548', '1018571', '101876',
       '1019', '101909', '102', '1020', '102003', '10202', '1021', '1022',
       '10223', '102233', '10225', '10226', '10230', '10232', '10234',
       '1024', '102463', '1025', '102568', '1026', '1027', '102774',
       '10279', '1028', '10281', '10284', '10285', '102857', '103',
       '1030', '1030534', '1030701', '1030713', '1030741', '1030745',
       '1030760', '1030776', '1030825', '1030977', '1031', '1031022',
      

In [70]:
ECH_customers[0:100]

array(['ECR-560156', 'ECR-10349400', 'ECR-29390542', 'ECR-10234417',
       'ECR-576739', 'ECR-11096900', 'ECR-10142636', 'ECR-551076',
       'ECR-854810', 'ECR-854908', 'ECR-854912', 'ECR-1226774',
       'ECR-100257593', 'ECR-11112752', 'ECR-10069255', 'ECR-10237448',
       'ECR-10338606', 'ECR-1228774', 'ECR-100240907', 'ECR-10333095',
       'ECR-25181811', 'ECR-100193831', 'ECR-11588067', 'ECR-1074454',
       'ECR-24349201', 'ECR-25742371', 'ECR-27227051', 'ECR-27915287',
       'ECR-30044003', 'ECR-33321129', 'ECR-10053975', 'ECR-1045814',
       'ECR-1226696', 'ECR-28285', 'ECR-353897', 'ECR-366468',
       'ECR-430560', 'ECR-434192', 'ECR-462249', 'ECR-521335',
       'ECR-537521', 'ECR-549911', 'ECR-562457', 'ECR-577786',
       'ECR-601275', 'ECR-927165', 'ECR-21213778', 'ECR-10179859',
       'ECR-10245864', 'ECR-10305954', 'ECR-10352887', 'ECR-10376710',
       'ECR-10401016', 'ECR-1152054', 'ECR-187769', 'ECR-20153845',
       'ECR-21289639', 'ECR-21289802', 'ECR-212917

In [33]:
# Common customers between ECH and other

len(np.intersect1d(ECH_customers, other_customers, assume_unique=True))

27666

In [34]:
# Common customers between ECH and Cancellations

len(np.intersect1d(ECH_customers, cancellation_customers, assume_unique=True))

8571

In [35]:
# Common customers between ECH and Churn activity

len(np.intersect1d(ECH_customers, churn_activity_customers, assume_unique=True))

25875

In [36]:
# Common customers between ECH and Churn risk customers

len(np.intersect1d(ECH_customers, churn_risks_customers, assume_unique=True))

39870

In [37]:
# Common customers between ECH and account_assignment_customers

len(np.intersect1d(ECH_customers, account_assignment_customers, assume_unique=True))

68030

In [38]:
# Common customers between ECH and usage_customers

len(np.intersect1d(ECH_customers, usage_customers, assume_unique=True))

21097

In [39]:
# Common customers between ECH and interactions_customers

len(np.intersect1d(ECH_customers, interactions_customers, assume_unique=True))

19120

In [40]:
# Common customers between ECH and NPS_customers

len(np.intersect1d(ECH_customers, NPS_customers, assume_unique=True))

7764

# Merge 

In [9]:
cust_journals = pd.merge(ECH, journals_contracts, how='inner', left_on='ecrid', right_on='SIS Id  (Agreement SIS)')

cust_others = pd.merge(ECH, other_contracts, how='inner', left_on='ecrid', right_on='SIS Id  (Agreement SIS)')

cust_cancelled = pd.merge(ECH, cancellations, how='inner', left_on='ecrid', right_on='SIS Id  (Agreement SIS)')

In [10]:
ECH['Country ISO'].value_counts(dropna=False)

US    50018
IN     8781
FR     8640
JP     7938
GB     5886
      ...  
WS        1
TM        1
SO        1
PM        1
MF        1
Name: Country ISO, Length: 208, dtype: int64

In [11]:
ECH.columns

Index(['ecrid', 'name', 'city', 'Country ISO', 'Region', 'consortium',
       'post_code', 'Classification'],
      dtype='object')

In [12]:
journals_contracts.columns

Index(['Agreement End Date', 'Agreement Number', 'Agreement Start Date',
       'Bookigns - Committed Print(Rep)',
       'Bookings - Final Net Price - Agent Discount Amount(Rep)',
       'Business Division (Agreement SIS)', 'Business Indicator',
       'Calculated New/Renewal', 'Country Name (Agreement SIS)', 'Division',
       'HQ SIS Id (Agreement SIS)', 'Invoice Date', 'Invoice Num',
       'Name  (Agreement SIS)', 'Parent Agreement Number', 'Payment Term',
       'Payment Term Description', 'Payment Term Type', 'Product Line Level 1',
       'Product Line Level 2', 'Product Line Level 3', 'Product Line Level 4',
       'Product Revenue Type', 'RSO', 'Renewal Exp Complete Date',
       'SIS Id  (Agreement SIS)', 'Saleable Product Name (Source)',
       'Sales Division (Agreement SIS)', 'Sales Type', 'Status',
       'Status Change Date', 'Subregion Grouping', 'Subscription End Date',
       'Subscription Start Date', 'Subscription Start Year', 'WIP Flag'],
      dtype='object')

## Journals

### Journals revenue by customers

In [14]:
pd.options.display.float_format = '${:,.0f}'.format


cust_journals.groupby(
    ['ecrid', 'name','Country ISO']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
     ).sort_values('revenue', ascending=False)

## also good idea to look at the the consortium number. same as consortium account.
## sis_id or consortia id

## negative values look like credit agreements
## customer doest want to sign agreement, but already charged 

,,,revenue,num_agreements
ecrid,name,Country ISO,,
ECR-490216,University Consortium for Digital Journals,FR,"$246,085,921",18
ECR-310775,Coordination of Higher Education Personnel Improvement,BR,"$207,604,102",118
ECR-380603,Collaboration of the Dutch university libraries and the Koninklijke Bibliotheek,NL,"$86,676,096",37
ECR-71655,Ohio Library and Information Network,US,"$86,269,246",21
ECR-393605,SUNYConnect,US,"$53,278,125",9
...,...,...,...,...
ECR-492708,AEP Books Inc,US,"$-48,808",0
ECR-1179134,University of Leeds Clinical Trials Research Unit,GB,"$-64,458",1
ECR-583967,European SocieTy for Radiotherapy and Oncology,BE,"$-66,474",1


### Journals Spend by country

In [15]:
cust_journals.groupby(
    ['Country ISO']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
     ).sort_values('revenue', ascending=False)

#just ensure Japan revenue is correct

,revenue,num_agreements
Country ISO,,
US,"$3,193,043,713",85233
JP,"$926,195,839",17187
CN,"$701,149,035",6836
GB,"$494,411,266",6984
FR,"$353,781,770",5429
...,...,...
TG,"$1,408",1
PG,$860,0
VI,$296,1


### Journals Revenue by product 

In [69]:
cust_journals.groupby(
    ['Product Line Level 2']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
     ).sort_values('revenue', ascending=False)

# check later

,revenue,num_agreements
Product Line Level 2,,
Journals Electronic Subscriptions,"$8,998,258,700.94",66139
Journals Print,"$634,085,520.87",105845
Journal Level Sales,"$454,128,151.21",39291
Journals Transactions,"$107,605,949.42",7084
Journals Backfiles,"$92,338,422.42",2698
...,...,...
Journals Print - One Off,"$63,530.53",1
Other Author Income,"$5,020.85",0
Language Editing Services,$587.39,0


In [16]:
cust_journals.groupby(
    ['ecrid', 'name','Product Line Level 2']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False)

,,,revenue,num_agreements
ecrid,name,Product Line Level 2,,
ECR-490216,University Consortium for Digital Journals,Journals Electronic Subscriptions,"$245,815,686",18
ECR-310775,Coordination of Higher Education Personnel Improvement,Journals Electronic Subscriptions,"$195,405,547",93
ECR-380603,Collaboration of the Dutch university libraries and the Koninklijke Bibliotheek,Journals Electronic Subscriptions,"$84,846,260",25
ECR-71655,Ohio Library and Information Network,Journals Electronic Subscriptions,"$82,842,911",20
ECR-393605,SUNYConnect,Journals Electronic Subscriptions,"$52,081,714",7
...,...,...,...,...
ECR-752,Linda Hall Library,Bulk Sales,"$-98,342",1
ECR-11266054,Schweitzer Fachinformationen oHG Central Journals Import,Bulk Sales,"$-149,389",1
ECR-1413,University of Essex,Journals Electronic Subscriptions,"$-195,188",6


### Customer journals revenue by year

In [60]:
cust_journals_year = cust_journals.groupby(
    ['ecrid', 'name','Subscription Start Year']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False).reset_index()


pd.pivot_table(cust_journals_year, 
               index=['ecrid','name'], 
               columns=['Subscription Start Year'], 
               values=['revenue'], 
               aggfunc=sum, fill_value=0, margins=True).sort_values(by=('revenue', 'All'), ascending=False)

revenue                                                                                           
Subscription Start Year                                                  2015           2016           2017           2018           2019           2020             All
ecrid       name                                                                                                                                                        
All                                                            $1,645,073,502 $1,734,860,943 $1,796,758,357 $1,821,696,336 $1,801,821,248 $1,653,042,049 $10,453,252,436
ECR-490216  University Consortium for Digital Journals            $39,256,562    $40,606,841    $41,735,252    $42,993,876    $41,586,168    $39,907,222    $246,085,921
ECR-310775  Coordination of Higher Education Personnel Impr...    $26,942,344    $25,549,101    $60,972,810    $30,129,666    $30,184,230    $33,825,950    $207,604,102
ECR-380603  Collaboration of the Dutch university libraries...    $13,701,712    $14,182,679    $14,489,812    $14,794,098    $14,794,098    $14,713,696     $86,676,096
ECR-71655   Ohio Library and Information Network                  $13,152,575    $14,102,916    $14,041,722    $14,483,386    $15,086,284    $15,402,363     $86,269,246
...                                                                       ...            ...            ...            ...            ...            ...             ...
ECR-492708  AEP Books Inc                                                  $0             $0       $-12,436       $-36,372             $0             $0        $-48,808
ECR-1179134 University of Leeds Clinical Trials Research Unit             $29            $31            $64       $-64,617            $35             $0        $-64,458
ECR-583967  European SocieTy for Radiotherapy and Oncology           $-23,553        $-1,795             $0       $-19,864       $-21,261             $0        $-66,474
ECR-411299  Canadian Institutes of Health Research                    $96,359      $-262,477      $-189,160        $12,585        $13,026        $13,612       $-316,054
ECR-2923    Hamamatsu University School of Medicine                 $-130,910      $-153,727      $-157,262      $-161,230      $-153,210             $0       $-756,339

[37229 rows x 7 columns]

In [59]:

product_journals_year = cust_journals.groupby(
    ['Product Line Level 2','Subscription Start Year']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False).reset_index()


pd.pivot_table(product_journals_year, 
               index=['Product Line Level 2'], 
               columns=['Subscription Start Year'], 
               values=['revenue'], 
               aggfunc=sum, fill_value=0, margins=True).sort_values(by=('revenue', 'All'), ascending=False)



revenue                                                                                           
Subscription Start Year                     2015           2016           2017           2018           2019           2020             All
Product Line Level 2                                                                                                                       
All                               $1,645,073,502 $1,734,860,943 $1,796,758,357 $1,821,696,336 $1,801,821,248 $1,653,042,049 $10,453,252,436
Journals Electronic Subscriptions $1,406,677,189 $1,478,352,390 $1,536,920,730 $1,536,399,233 $1,511,200,168 $1,528,708,990  $8,998,258,701
Journals Print                      $123,074,090   $128,425,866   $124,025,066   $117,593,573   $105,794,723    $35,172,203    $634,085,521
Journal Level Sales                  $61,872,920    $70,459,103    $77,361,368    $80,133,764    $80,639,117    $83,661,878    $454,128,151
Journals Transactions                $19,195,410    $21,079,686    $19,686,255    $21,993,408    $24,487,058     $1,164,132    $107,605,949
...                                          ...            ...            ...            ...            ...            ...             ...
Journals Print - One Off                    $375         $8,015         $8,533        $35,646        $10,961             $0         $63,531
Other Author Income                           $0             $0             $0         $5,021             $0             $0          $5,021
Language Editing Services                     $0             $0             $0           $302           $285             $0            $587
Conferences - Delegate                      $226             $0             $0             $0             $0             $0            $226
Journals Subscriptions Electronic      $-195,553      $-145,560       $-27,465       $-36,372           $331             $0       $-404,619

[27 rows x 7 columns]

## Other products

### Other products Revenue by customer

In [79]:
cust_others.groupby(
    ['ecrid', 'name','Country ISO']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
     ).sort_values('revenue', ascending=False)

,,,revenue,num_agreements
ecrid,name,Country ISO,,
ECR-10425118,Elsevier Ltd Kidlington Corporate Office,GB,"$213,117,362.37",2
ECR-79671,Amazon Com Books,US,"$127,993,160.99",1
ECR-10408016,Login Canada,CA,"$70,476,435.08",1
ECR-452700,Kettering Medical Center,US,"$61,667,031.07",10
ECR-310775,Coordination of Higher Education Personnel Improvement,BR,"$50,928,117.53",83
...,...,...,...,...
ECR-936052,Sentara Healthcare Inc,US,"$-61,633.00",0
ECR-10317219,Amazon.com Inc,US,"$-227,387.58",1
ECR-22572844,UBS Publishers Distributors Pvt Ltd,IN,"$-366,306.61",1


### Revenue by product

In [71]:
cust_others.groupby(
    ['Product Line Level 2']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
     ).sort_values('revenue', ascending=False)

# print might not have agreement
# exclude HS and CS
# Stick to RSS

,revenue,num_agreements
Product Line Level 2,,
Nursing & Health Professionals P_ELP,"$941,833,108.05",1
CK Physician,"$876,364,687.40",10055
SCOPUS,"$811,055,334.25",13178
Reaxys,"$422,539,203.93",5914
eBooks,"$329,351,367.03",33769
...,...,...
Journals Pharma Print Other,"$4,352.31",0
Books Electronic,$626.84,1
Books Other,$5.66,1


### customer revenue by year

In [61]:
cust_others_year = cust_others.groupby(
    ['ecrid', 'name','Subscription Start Year']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False).reset_index()


pd.pivot_table(cust_others_year, 
               index=['ecrid','name'], 
               columns=['Subscription Start Year'], 
               values=['revenue'], 
               aggfunc=sum, fill_value=0, margins=True).sort_values(by=('revenue', 'All'), ascending=False)

revenue                                                                                        
Subscription Start Year                                         2015.0         2016.0         2017.0         2018.0         2019.0       2020.0            All
ecrid        name                                                                                                                                             
All                                                     $1,105,620,256 $1,135,003,226 $1,255,632,845 $1,221,725,724 $1,253,874,374 $712,249,909 $6,684,106,333
ECR-10425118 Elsevier Ltd Kidlington Corporate Office      $46,415,834    $41,778,226    $48,568,096    $37,849,305    $38,505,901           $0   $213,117,362
ECR-79671    Amazon Com Books                              $13,425,834    $20,677,682    $32,101,122    $33,760,449    $28,028,074           $0   $127,993,161
ECR-10408016 Login Canada                                  $16,850,776    $12,859,826    $13,455,232    $15,946,631    $11,363,971           $0    $70,476,435
ECR-452700   Kettering Medical Center                          $47,153        $63,913    $60,775,799       $242,437       $285,009     $252,720    $61,667,031
...                                                                ...            ...            ...            ...            ...          ...            ...
ECR-936052   Sentara Healthcare Inc                                 $0             $0             $0       $-61,633             $0           $0       $-61,633
ECR-10317219 Amazon.com Inc                                  $-189,507       $-23,232       $-10,006        $-1,038        $-3,605           $0      $-227,388
ECR-22572844 UBS Publishers Distributors Pvt Ltd             $-126,927             $0      $-236,301        $-3,078             $0           $0      $-366,307
ECR-10366988 Dar Al Jadawel Publishing and Distribution        $83,667      $-675,978        $41,703         $6,461        $34,595           $0      $-509,552
ECR-10066097 DHL International UK Ltd Rushden                $-638,036      $-610,498      $-437,532      $-445,617      $-330,764           $0    $-2,462,448

[27615 rows x 7 columns]

### product revenue by year

In [62]:
product_others_year = cust_others.groupby(
    ['Product Line Level 2','Subscription Start Year']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False).reset_index()


pd.pivot_table(product_others_year, 
               index=['Product Line Level 2'], 
               columns=['Subscription Start Year'], 
               values=['revenue'], 
               aggfunc=sum, fill_value=0, margins=True).sort_values(by=('revenue', 'All'), ascending=False)



revenue                                                                                        
Subscription Start Year                      2015.0         2016.0         2017.0         2018.0         2019.0       2020.0            All
Product Line Level 2                                                                                                                       
All                                  $1,105,620,256 $1,135,003,226 $1,255,632,845 $1,221,725,724 $1,253,874,374 $712,249,909 $6,684,106,333
Nursing & Health Professionals P_ELP   $196,536,459   $184,540,263   $204,915,834   $183,277,276   $172,563,275           $0   $941,833,108
CK Physician                           $113,537,947   $127,094,910   $198,667,648   $144,865,218   $153,029,665 $139,169,299   $876,364,687
SCOPUS                                  $99,840,758    $98,870,077   $106,599,296   $115,421,400   $121,136,980 $123,714,761   $665,583,271
Reaxys                                  $55,743,349    $56,111,713    $57,835,545    $58,416,785    $60,305,973  $58,467,372   $346,880,737
...                                             ...            ...            ...            ...            ...          ...            ...
Journals Pharma Print Other                      $0             $0             $0         $4,352             $0           $0         $4,352
Books Electronic                          $-112,561        $31,655      $-140,436        $25,618       $196,351           $0           $627
Books Other                                      $1             $1             $1             $1             $1           $0             $6
S&T Books                                        $0             $0             $0             $0             $0           $0             $0
Books Print                                  $3,128      $-284,344       $-84,531         $5,029        $28,693           $0      $-332,025

[119 rows x 7 columns]

## Cancelled contracts

### Cancelled revenue by customer

In [88]:
cust_cancelled.groupby(
    ['ecrid', 'name', 'Country ISO']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
     ).sort_values('revenue', ascending=False)

,,,revenue,num_agreements
ecrid,name,Country ISO,,
ECR-1004,University of California System,US,"$10,746,571",5
ECR-375272,Konsortium Baden-Wurttemberg,DE,"$6,013,895",1
ECR-44401,Hungarian Academy of Sciences,HU,"$5,164,912",1
ECR-928880,Centre for Research On Scientific and Technical Information,DZ,"$4,856,530",4
ECR-339079,Ministry of Higher Education Scientific Research and Technology and Information and Communication Technologies,TN,"$3,949,677",1
...,...,...,...,...
ECR-365981,Health and Safety Authority,IE,$-0,1
ECR-417308,Forest University,CO,$-0,2
ECR-737806,University Colleges of Columbia,CO,$-0,1


### Cancelled revenue by product

In [82]:
cust_cancelled.groupby(
    ['Product Line Level 2']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
     ).sort_values('revenue', ascending=False)

,revenue,num_agreements
Product Line Level 2,,
Journals Electronic Subscriptions,"$173,465,915.16",3811
CK Physician,"$25,987,050.38",646
SCOPUS,"$23,727,729.97",842
Reaxys,"$17,066,727.11",451
Journal Level Sales,"$12,309,281.50",2313
...,...,...
SSRN,"$332,714.76",107
Life Sciences Other,"$287,551.14",8
CK Med Ed,"$137,429.47",8


### Cancellations by customer and year

In [89]:
cust_cancelled_year = cust_cancelled.groupby(
    ['ecrid', 'name', 'Cancellation Year']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False).reset_index()


pd.pivot_table(cust_cancelled_year, 
               index=['ecrid','name'], 
               columns=['Cancellation Year'], 
               values=['revenue'], 
               aggfunc=sum, fill_value=0, margins=True).sort_values(by=('revenue', 'All'), ascending=False)

revenue                                                                                  
Cancellation Year                                                    2015        2016        2017        2018        2019        2020     2021          All
ecrid      name                                                                                                                                            
All                                                           $36,958,283 $42,878,972 $69,302,257 $95,173,482 $70,363,467 $14,975,093 $142,444 $329,793,998
ECR-1004   University of California System                             $0    $200,000          $0          $0 $10,546,571          $0       $0  $10,746,571
ECR-375272 Konsortium Baden-Wurttemberg                                $0          $0          $0  $6,013,895          $0          $0       $0   $6,013,895
ECR-44401  Hungarian Academy of Sciences                               $0          $0          $0          $0  $5,164,912          $0       $0   $5,164,912
ECR-928880 Centre for Research On Scientific and Technical...    $856,530          $0  $4,000,000          $0          $0          $0       $0   $4,856,530
...                                                                   ...         ...         ...         ...         ...         ...      ...          ...
ECR-365981 Health and Safety Authority                                 $0          $0         $-0          $0          $0          $0       $0          $-0
ECR-417308 Forest University                                          $-0          $0          $0          $0          $0          $0       $0          $-0
ECR-737806 University Colleges of Columbia                             $0         $-0          $0          $0          $0          $0       $0          $-0
ECR-463151 Xinjiang University                                        $-0          $0          $0          $0          $0          $0       $0          $-0
ECR-566978 HS EMEA/LA Asklepios Kliniken Gmbh                          $0          $0          $0          $0          $0         $-0       $0          $-0

[8572 rows x 8 columns]

### Product cancellations by year

In [90]:
product_cancelled_year = cust_cancelled.groupby(
    ['Product Line Level 2','Cancellation Year']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False).reset_index()


pd.pivot_table(product_cancelled_year, 
               index=['Product Line Level 2'], 
               columns=['Cancellation Year'], 
               values=['revenue'], 
               aggfunc=sum, fill_value=0, margins=True).sort_values(by=('revenue', 'All'), ascending=False)

revenue                                                                                  
Cancellation Year                        2015        2016        2017        2018        2019        2020     2021          All
Product Line Level 2                                                                                                           
All                               $36,958,283 $42,878,972 $69,302,257 $95,173,482 $70,363,467 $14,975,093 $142,444 $329,793,998
Journals Electronic Subscriptions $14,304,948 $15,702,864 $40,462,644 $59,094,604 $37,172,007  $6,619,770 $109,078 $173,465,915
CK Physician                       $3,515,455  $4,757,864  $5,960,471  $5,825,478  $5,822,804    $104,979       $0  $25,987,050
SCOPUS                             $4,532,807  $5,148,291  $3,672,445  $4,487,878  $4,253,345  $1,632,964       $0  $23,727,730
Reaxys                             $2,921,846  $2,189,584  $2,889,784  $3,584,287  $3,627,993  $1,853,233       $0  $17,066,727
...                                       ...         ...         ...         ...         ...         ...      ...          ...
SSRN                                       $0          $0      $4,799    $184,973    $140,291      $2,652       $0     $332,715
Life Sciences Other                   $33,684     $84,228    $102,737          $0     $66,903          $0       $0     $287,551
CK Med Ed                                  $0          $0          $0      $6,399    $131,031          $0       $0     $137,429
Journals Legacy                            $0    $103,227          $0          $0          $0          $0       $0     $103,227
Journals Other                             $0          $0     $45,885          $0          $0          $0       $0      $45,885

[35 rows x 8 columns]

In [91]:
product_cancelled_year = cust_cancelled.groupby(
    ['Product Line Level 3','Cancellation Year']
).agg(revenue=("Bookings - Final Net Price - Agent Discount Amount(Rep)", sum),
      num_agreements=('Agreement Number', pd.Series.nunique)
    ).sort_values('revenue', ascending=False).reset_index()


pd.pivot_table(product_cancelled_year, 
               index=['Product Line Level 3'], 
               columns=['Cancellation Year'], 
               values=['revenue'], 
               aggfunc=sum, fill_value=0, margins=True).sort_values(by=('revenue', 'All'), ascending=False)

revenue                                                                                  
Cancellation Year                      2015        2016        2017        2018        2019        2020     2021          All
Product Line Level 3                                                                                                         
All                             $36,958,283 $42,878,972 $69,302,257 $95,173,482 $70,363,467 $14,975,093 $142,444 $329,793,998
E-fees                           $5,956,968  $5,220,927 $23,147,395 $28,576,178 $20,727,686  $2,797,317   $3,137  $86,429,607
Journal Collections              $7,606,413  $9,321,215 $16,122,742 $28,170,217 $15,639,589  $3,368,430 $105,736  $80,334,341
Scopus                           $4,532,807  $5,148,291  $3,657,445  $4,412,878  $4,247,313  $1,614,573       $0  $23,613,307
ClinicalKey Global               $3,325,261  $4,402,658  $4,931,628  $5,265,285  $4,700,726     $51,352       $0  $22,676,910
...                                     ...         ...         ...         ...         ...         ...      ...          ...
Clinical Pharmacology RD 1               $0          $0          $0          $0          $0          $0       $0           $0
MDC Specialty Edition                    $0          $0          $0          $0          $0          $0       $0           $0
MDC ANZ Standard Subscr                  $0          $0          $0          $0          $0          $0       $0           $0
MD Consult Link                          $0          $0          $0          $0          $0          $0       $0           $0
Natural Medicines Database Link          $0          $0          $0          $0          $0          $0       $0           $0

[160 rows x 8 columns]